# 4.16.39 Data Distribution

### Visualising the distribution of your data

The statistics we’ve covered in our previous class summarise well the data to a single number that describes the location or the dispersion of our data series. It is also useful to visually explore the complete distribution of the data.

There are several ways to visualise the distribution of a dataset, we will see the most widespread and useful options available. 

<img src="img/distr-charts.png" width="600">

### The Box and Whisker Plot

Also known as Box Plot, a Box and Whisker Plot is a very useful and fast way of visually displaying the data distribution (locality, dispersion and skewness) through their quartiles. It was first introduced by John Tukey in 1970, who later published on the subject in his book *Exploratory Data Analysis* in 1977.

<img src="img/boxplot-anatomy.png" width="700">

Let's see **how to read** a Box Plot by describing its anatomy: 

- the **box in the middle** is defined by three vertical lines that indicate (from left to right) the 25th, 50th (median) and 75th percentile (that is, the 1st, 2nd and 3rd quartile);
- the horizontal lines extending parallel from the boxes are known as the **whiskers**, which are used to indicate variability outside the upper and lower quartiles; 
- the whiskers extend to the furthest point beyond the box, except that they will not go beyond **1.5 times the IQR** (depending on the software used, this rule may change); 
- very **extreme values** and **potential outliers** are sometimes plotted as individual dots that are in-line with whiskers; 
- box plots can be drawn either vertically or horizontally.

Although Box Plots may seem simple in comparison to a Histogram or Density Plot, their advantage is that they are more compact and explicit, which comes in handy when comparing data across different groups or categories.

So **what can we understand** about our data when looking at a Box Plot? 

- with a single glance we can get an idea of what the key statistics are, such as the median, IQR, quartiles and more; 
- how clustered or dispersed the data is;
- if there are any outliers and what their values are; 
- if the data is symmetrical or skewed and if so, in what direction. 

Let's see a **practical example**. We've briefly seen the `planets` dataset in the homework assignment, whose data come from NASA's [Exoplanet Exploration program](https://exoplanets.nasa.gov/discovery/exoplanet-catalog/). The dataset includes some recorded metrics like `orbital_period`, `mass` and `distance`. Each observation is also identified by its `method` of discovery (check out [this page](https://exoplanets.nasa.gov/discovery/how-we-find-and-characterize/) for more details on that). 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
planets = sns.load_dataset('planets')
print('DataFrame size:', planets.shape)
planets.head(3)

In the homework assignment we looked at some descriptive statistics for the `distance` metric as a whole. Let's briefly recap what we discovered in the homework and then visualise the same information using a box plot. 

In [ ]:
planets.distance.describe()

In [ ]:
sns.stripplot(x = 'distance', data = planets)

Let's create a box plot of the `distance` metric: 

In [ ]:
sns.boxplot(x='distance', data=planets)

It doesn't really look like what we saw in the diagram above; in fact, it is all squeezed to the left. This is because 75% of the data is lies between 1.35 and 178.5 (see min and 75% in the output of the `.describe()` method), all the rest are extreme values that skew the data creating a long tail on the right-side of the distribution. 

Although it is not a problem per se, it would make sense to describe the `distance` variable **within each group** of the `method` used. We can do that by grouping the DataFrame by the `method` used, aggregating by the `distance` metric and finally calling the `.describe()` method on the resulting DataFrame. 

In [ ]:
planets.groupby('method')['distance'].describe()

We can also use the `sns.stripplot()` function to visualise each group by including `y = 'method'` as a parameter:

In [ ]:
sns.stripplot(x = 'distance', y = 'method', data = planets)

Similarly, we can generate a series of box plots by including the same `y = 'method'` parameter in the `sns.boxplot()` function call: 

In [ ]:
sns.boxplot(x = 'distance', y = 'method', data = planets, fliersize=3, linewidth=1)

Let's look at a single box plot, particularly, let's look at the "Radial Velocity" `method`: 

In [ ]:
sns.boxplot(x = 'distance', y = 'method', data = planets[planets.method=='Radial Velocity'], fliersize=3, linewidth=1)

In [ ]:
sns.swarmplot(x = 'distance', y = 'method', data = planets[planets.method=='Radial Velocity'], size = 4)

In [ ]:
sns.boxplot(x = 'distance', y = 'method', data = planets[planets.method=='Radial Velocity'], fliersize=0, linewidth=2)
sns.swarmplot(x = 'distance', y = 'method', data = planets[planets.method=='Radial Velocity'], size = 3.5, color = 'orange')

Let's use the `year` of discovery variable to create a new `century` column that divides our observations in "20th" and "21st" cetury so that we can use this new categorical variable to check for differences in the two distributions.  

In [ ]:
planets['century'] = ['20th' if el < 2000 else '21th' for el in planets.year]

In [ ]:
planets.head()

In [ ]:
sns.boxplot(x = 'distance', y = 'method', hue = 'century', data = planets[planets.method=='Radial Velocity'])

It looks like objects discovered in the 20th Century have a shorter distance from earth; this makes sense, since telescope's technology and ability to look further has drastically increased in the last decades. 

#### Exercise

Jupiter is by far the most massive planet in the Solar System. It is approximately 2.5 times as massive as all of the other planets in the Solar System combined. **Jupiter mass** is the unit of mass equal to the total mass of the planet Jupiter and it is used to describe masses of gas giants, such as the outer planets and exoplanets. 

The mass variable in our dataset is expressed in Jupiters, so it could be interesting to create a new variable `jupiters` that classifies our observations in whether they have a mass greater or inferior to 1 Jupiter. 

In [ ]:
planets.mass.describe()

In [ ]:
planets['jupiters'] = ['up to 1 Jupiter' if el<=1 else 'over 1 Jupiter' for el in planets.mass]

In [ ]:
planets.head()

In [ ]:
sns.boxplot(x = 'distance', y = 'century', hue = 'jupiters', data = planets[planets.method=='Radial Velocity'], palette="Set3")

### Frequency tables and  Histograms

A **frequency table** of a variable divides up the variable range into equally spaced segments (also known as **bins**), and counts how many observations fall in each segment. 

> Similarly to percentiles, also frequency tables summarize the data by creating bins. However, while quartiles and deciles will have the same count of observations in each bin (but the bin sizes will be different), the frequency table will have different counts in each bins (but equally spaced bins).

Let's take the `distance` variable from our `planets` dataset and create a new column `bins` that divides the metric's range in 10 equal parts. The `pd.cut()` function from the `pandas` library allows us to do just that: 

In [ ]:
planets['distance'].describe()

In [ ]:
planets['bins'] = pd.cut(x=planets['distance'], bins=10)
# To fix the negative bins issue: 
#first_I = planets['bins'].cat.categories[0]
#new_I = pd.Interval(min(planets['distance']), first_I.right)
#planets['bins'] = planets['bins'].cat.rename_categories({first_I: new_I})

In [ ]:
planets.sample(3)

At this point, we can generate the **frequency table** using the `.value_counts()` method on the `bins` column:

In [ ]:
planets.bins.cat.categories.tolist()

In [ ]:
planets.bins.value_counts()

A **histogram** is a way to visualize a frequency table, with bins on the horizontal axis and the count of observations on the vertical axis. Histograms are plotted such that:

- all bins have the **same width**;
- the **number of bins** is decided by the analyst;
- **empty bins** are kept in the chart.

Technically, we can use a bar plot to visualise the frequency table: 

In [ ]:
sns.barplot(x = planets.bins.value_counts().index, 
            y = planets.bins.value_counts().values)

But luckily for us, the `seaborn` library comes with a handy `sns.histplot()` [function](https://seaborn.pydata.org/generated/seaborn.histplot.html):

In [ ]:
sns.histplot(x = 'distance', data = planets, bins = 10)

The `stat` parameter allows you to choose between different statistics, such as `count` or `frequency` to be used as the y-axis metric; for instance, the frequency option shows the number of observations divided by the number of bin.

In [ ]:
sns.histplot(x = 'distance', data = planets, bins = 10, stat = 'frequency')

In statistics, location and variability are referred to as the **first and second moments** of a distribution. The **third and fourth moments** are called [skewness](https://en.wikipedia.org/wiki/Skewness) and [kurtosis](https://en.wikipedia.org/wiki/Kurtosis): 

- **skewness** is a measure of lack of symmetry, it refers to whether the bulk of the data is shifted to larger or smaller values; 
- **kurtosis** is a measure of whether the data tends to have heavy tails (that is, to have extreme values or outliers). 

Although it is possible to measure skewness and kurtosis, it is common practice to evaluate these two characteristics of the data using visual aids such as box plots, histograms and density plots. 

<img src="img/skewness-kurtosis.png" width="900">

As an example, let's plot the histogram of the `mass` column:

In [ ]:
sns.histplot(x = 'mass', data = planets, bins = 10, stat = 'frequency')
print('Skewness:', stats.skew(planets.mass.dropna()))
print('Kurtosis:', stats.kurtosis(planets.mass.dropna()))

From a quick visual analysis we can conclude that the distribution of the `mass` variable is **positively skewed** and **leptokurtic**:

- it is clear that is is not symmetric, in fact the bulk of the data lays on the left side of the x-axis; in such cases we say that the distribution is **postively skewed**, as opposed to a **negatively skewed** distribution (which would have the hump on the right-side. The Normal distribution (which we'll cover in a future class) is **symmetric** and therefore has a **skewness of zero**; 
- the kurtosis can be detected by how "tall" or "flat" the peak of the distribution is; a "slim and tall" distribution (like the one we're analysing now) will have a high kurtosis and is called **leptokurtic**, as opposed to a "fat and short" distribution that will have a low kurtosis and is called **platikurtic**. The Normal distribution is said to be **mesokurtic** and, with a **kurtosis of 3** is somewhere in between. 

### Density plots

A **density plot**, can be thought of as a smoothed version of a histogram and shows the distribution of data values as a continuous line. It is typically computed directly from the data using a particular function known technically as a [kernel density estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation). 

Let's load the `penguins` dataset from the `seaborn` library and plot a histogram together with a density plot of the `bill_length_mm` variable using the `kde=True` parameter in the `sns.histplot()` function: 

In [ ]:
peng = sns.load_dataset('penguins')
peng.head(3)

In [ ]:
sns.histplot(x = 'bill_length_mm', data = peng, bins = 10, stat = 'probability', kde = True)
plt.show()

The distribution of the `bill_length_mm` variable has an interesting shape, it looks like the back of a camel with those two humps. Technically, we say that this is a [bimodal distribution](https://en.wikipedia.org/wiki/Multimodal_distribution), which means (as the word suggests) that the data has two modes. This could be a first red flag that maybe we should **look at this data at a different level of aggregation**, for instance checking for differences between `species` or `sex`. 

Let's use this cue to look at the `bill_length_mm` across the three different `species` of penguins availabe; this time we'll use the `sns.kdeplot()` function:

In [ ]:
sns.kdeplot(x = 'bill_length_mm', data = peng, hue = 'species', fill = True)
plt.show()

In [ ]:
sns.kdeplot(x = 'bill_length_mm', data = peng[peng.species=='Chinstrap'], hue = 'sex', fill = True)
plt.show()

### Bivariate distributions

So far we have looked at **univariate distributions**, that is, how the data of a single variable behaves in its own range of variation. It may be interesting to look at how the data from this single variable interacts with a second variable; just like when we use a scatterplot to visualise the relationship between two bariables, we can compute what is technically known as the [joint probability distribution](https://en.wikipedia.org/wiki/Joint_probability_distribution) between these two variables in order to evaluate the probability distribution on all possible pairs of outputs. 

This is known as a **bivariate distribution**; the `sns.histplot()` and `sns.kdeplot()` functions support bivariate plots by simpling adding a `y=` parameter; let's have a look at the bivariate distribution between the `bill_length_mm` and the `bill_depth_mm` variables: 

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(13, 5)

sns.histplot(x = 'bill_length_mm', y = 'bill_depth_mm', data = peng, ax = ax[0]).set_title("Visualising the bivariate distribution via Histogram")
sns.kdeplot(x = 'bill_length_mm', y = 'bill_depth_mm', data = peng, ax = ax[1]).set_title("Visualising the bivariate distribution via Density plot")
plt.show()

Similarlt to what we did with the univariate distribution, we can use the `hue=` parameter to group the data and produce separate distribution for each aggregation: 

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(13, 5)

sns.histplot(x = 'bill_length_mm', y = 'bill_depth_mm', data = peng, hue="species", ax = ax[0]).set_title("Visualising the grouped bivariate distribution via Histogram")
sns.kdeplot(x = 'bill_length_mm', y = 'bill_depth_mm', data = peng, hue="species", ax = ax[1]).set_title("Visualising the grouped bivariate distribution via Density plot")
plt.show()

Thanks to the `sns.jointplot()` function it is also possible to investigate the relationship between two variables using a standard scatterplot and, jointly, visualise the data distribution by adding a histogram or a density plot on the margins: 

In [ ]:
sns.jointplot(data=peng, x="bill_length_mm", y="bill_depth_mm", hue = 'species', height=8)
plt.show()

Finally, the `sns.rugplot()` function can be used to **add "rugs" on the side of another plot**, which allows us to get a general idea on how the data is distributed within each axis/variable. This is very useful in scatterplots, especially in cases where there are a lot of observations and their overlap makes it harder to visualise areas of higher data density. 

Let's see an example of how to add a `sns.rugplot()` to a `sns.scatterplot()`:

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(18, 8)

sns.scatterplot(data=peng, x="bill_length_mm", y="bill_depth_mm", ax=ax[0]).set_title("Bivariate relationship via Scatterplot and Rugplot")
sns.rugplot(data=peng, x="bill_length_mm", y="bill_depth_mm", ax=ax[0])

sns.scatterplot(data=peng, x="bill_length_mm", y="bill_depth_mm", hue='species', ax=ax[1]).set_title("Bivariate grouped relationship via Scatterplot and Rugplot")
sns.rugplot(data=peng, x="bill_length_mm", y="bill_depth_mm", hue='species', ax=ax[1])

plt.show()